In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()  # for plot styling
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
import itertools
from sklearn.metrics import silhouette_score, silhouette_samples, davies_bouldin_score
from sklearn import preprocessing
from sklearn.decomposition import PCA

In [2]:
files = ['./data/BattlesStaging_01012021_WL_tagged.csv', './data/BattlesStaging_01022021_WL_tagged.csv', 
         './data/BattlesStaging_01022021_WL_tagged.csv', './data/BattlesStaging_01042021_WL_tagged.csv',
         './data/battlesStaging_12272020_WL_tagged.csv', './data/battlesStaging_12282020_WL_tagged.csv',
         './data/BattlesStaging_12292020_WL_tagged.csv', './data/BattlesStaging_12302020_WL_tagged.csv',
         './data/BattlesStaging_12312020_WL_tagged.csv',]
df = pd.concat([pd.read_csv(f) for f in files], ignore_index=True)
df.head()

,Unnamed: 0,battleTime,arena.id,gameMode.id,average.startingTrophies,winner.tag,winner.startingTrophies,winner.trophyChange,winner.crowns,winner.kingTowerHitPoints,...,loser.cards.list,loser.totalcard.level,loser.troop.count,loser.structure.count,loser.spell.count,loser.common.count,loser.rare.count,loser.epic.count,loser.legendary.count,loser.elixir.average
0,0,2020-12-31 21:02:12+00:00,54000050.0,72000006.0,5363.0,#PVLPJP2Y,5372.0,28.0,2.0,4145.0,...,"[26000000, 26000015, 26000023, 27000004, 28000...",104,3,1,4,1,1,4,2,3.500
1,1,2020-12-31 21:02:15+00:00,54000050.0,72000006.0,5407.0,#8PRLRYYCV,5409.0,29.0,1.0,5304.0,...,"[26000023, 26000027, 26000037, 26000046, 26000...",104,6,1,1,0,1,2,5,4.250
2,2,2020-12-31 21:02:45+00:00,54000050.0,72000006.0,5741.0,#2G8LQRCG,5749.0,28.0,2.0,5762.0,...,"[26000022, 26000027, 26000028, 26000041, 26000...",104,7,0,1,4,2,1,1,4.125
3,3,2020-12-31 21:03:13+00:00,54000050.0,72000006.0,4307.0,#Y9QL09VGV,4316.0,28.0,2.0,4392.0,...,"[26000012, 26000027, 26000031, 26000033, 26000...",80,6,1,1,2,1,2,3,3.750
4,4,2020-12-31 21:03:17+00:00,54000050.0,72000006.0,5776.5,#9RRYG9P9U,5783.0,28.0,3.0,5832.0,...,"[26000010, 26000011, 26000021, 26000037, 26000...",104,5,1,2,2,4,0,2,3.250


In [3]:
df["loser.cards.list"]

0           [26000000, 26000015, 26000023, 27000004, 28000...
1           [26000023, 26000027, 26000037, 26000046, 26000...
2           [26000022, 26000027, 26000028, 26000041, 26000...
3           [26000012, 26000027, 26000031, 26000033, 26000...
4           [26000010, 26000011, 26000021, 26000037, 26000...
                                  ...                        
21013688    [26000006, 26000007, 26000008, 26000012, 26000...
21013689    [26000004, 26000036, 26000042, 26000046, 26000...
21013690    [26000011, 26000015, 26000024, 26000025, 26000...
21013691    [26000004, 26000007, 26000008, 26000015, 26000...
21013692    [26000000, 26000026, 26000041, 27000003, 28000...
Name: loser.cards.list, Length: 21013693, dtype: object

In [4]:
df["winner.cards.list"]

0           [26000003, 26000004, 26000006, 26000008, 26000...
1           [26000000, 26000010, 26000026, 26000040, 26000...
2           [26000010, 26000042, 26000044, 26000055, 26000...
3           [26000016, 26000023, 26000026, 26000041, 28000...
4           [26000010, 26000014, 26000021, 26000030, 26000...
                                  ...                        
21013688    [26000007, 26000010, 26000016, 26000035, 26000...
21013689    [26000011, 26000012, 26000017, 26000021, 26000...
21013690    [26000011, 26000017, 26000022, 26000033, 26000...
21013691    [26000011, 26000042, 26000055, 26000061, 26000...
21013692    [26000014, 26000018, 26000032, 26000038, 26000...
Name: winner.cards.list, Length: 21013693, dtype: object

In [5]:
df_final = df[["winner.cards.list", "loser.cards.list", "winner.crowns"]]

In [6]:
df_final.head()

,winner.cards.list,loser.cards.list,winner.crowns
0,"[26000003, 26000004, 26000006, 26000008, 26000...","[26000000, 26000015, 26000023, 27000004, 28000...",2.0
1,"[26000000, 26000010, 26000026, 26000040, 26000...","[26000023, 26000027, 26000037, 26000046, 26000...",1.0
2,"[26000010, 26000042, 26000044, 26000055, 26000...","[26000022, 26000027, 26000028, 26000041, 26000...",2.0
3,"[26000016, 26000023, 26000026, 26000041, 28000...","[26000012, 26000027, 26000031, 26000033, 26000...",2.0
4,"[26000010, 26000014, 26000021, 26000030, 26000...","[26000010, 26000011, 26000021, 26000037, 26000...",3.0


In [7]:
df_final.shape

(21013693, 3)

In [8]:
df_final.to_csv("final.csv", index=False)